# **Instalar Librerias y definición de datos.**

In [2]:
#!pip install scikit-surprise
#!pip install lightfm

###############################################################################
# pemalink: https://github.com/JossueGG/tec_bigdata_equipo17/blob/30ae4d31c8f6bf2b6be5bf47f2294a16199b19c4/ratings_small.csv

# URL del archivo CSV en git o un contenedor
url = 'https://raw.githubusercontent.com/JossueGG/tec_bigdata_equipo17/30ae4d31c8f6bf2b6be5bf47f2294a16199b19c4/ratings_small.csv?token=GHSAT0AAAAAACSA6SWKKTOK5AYQCXS4PTAAZSOELKQ'


# ***Entrega 1***

In [ ]:
#!pip install scikit-surprise
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Rango de las calificaciones
reader = Reader(rating_scale=(1, 5))

# Número de recomendaciones a desplegar
rec = 5

###############################################################################

# Cargar los datos desde la URL
df = pd.read_csv(url)

# Crear un Dataset de Surprise
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Entrenando el modelo
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Obtener el nombre de la película a partir de su ID
movie_titles = df[['movieId', 'title']].drop_duplicates().set_index('movieId')['title'].to_dict()

# Función para obtener las mejores recomendaciones para un usuario dado
def get_top_recommendations(user_id, n=rec):
    items = []
    # Obtener todos los ítems que el usuario no ha calificado
    items_not_rated_by_user = [item for item in range(1, trainset.n_items + 1)
                               if item not in trainset.ur[user_id]]
    # Obtener la predicción de calificación para cada ítem no calificado
    for item_id in items_not_rated_by_user:
        predicted_rating = model.predict(user_id, item_id).est
        items.append((item_id, predicted_rating))
    # Ordenar las predicciones y devolver las mejores n recomendaciones
    top_items = sorted(items, key=lambda x: x[1], reverse=True)[:n]
    return [(item[0], movie_titles[item[0]]) for item in top_items]


###############################################################################
# Ejemplo de uso, recomendación para el usuario dado
###############################################################################
user_id = 1
top_recommendations = get_top_recommendations(user_id)
print("Top recomendaciones para el usuario:", user_id)
for rank, (movie_id, title) in enumerate(top_recommendations, 1):
    print(f"{rank}. {title} (ID: {movie_id})")

# Evaluar el modelo con 5-fold cross-validation
print("\n\n\nEvaluación del modelo:")
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Top recomendaciones para el usuario: 1
1. The Shawshank Redemption (ID: 318)
2. All About Eve (ID: 926)
3. Raging Bull (ID: 1228)
4. Stand by Me (ID: 1259)
5. Modern Times (ID: 3462)



Evaluación del modelo:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8935  0.8979  0.8943  0.9013  0.8937  0.8961  0.0030  
MAE (testset)     0.6893  0.6919  0.6878  0.6953  0.6857  0.6900  0.0033  
Fit time          1.46    1.99    1.65    1.52    1.49    1.62    0.20    
Test time         0.24    0.20    0.22    0.24    0.11    0.20    0.05    


# **Entrega 2**

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, GridSearchCV
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
from scipy.sparse import coo_matrix
import numpy as np

# Rango de las calificaciones
reader = Reader(rating_scale=(1, 5))

# Número de recomendaciones a desplegar
rec = 5

# Cargar los datos desde la URL
df = pd.read_csv(url)

# Crear un Dataset de Surprise
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Búsqueda de hiperparámetros para SVD
param_grid = {
    'n_factors': [20, 50, 100, 150],
    'n_epochs': [10, 20, 30, 40, 50],
    'lr_all': [0.002, 0.005, 0.01, 0.02],
    'reg_all': [0.02, 0.05, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Mejor modelo SVD
best_model = gs.best_estimator['rmse']
best_model.fit(data.build_full_trainset())

# Obtener el nombre de la película a partir de su ID
movie_titles = df[['movieId', 'title']].drop_duplicates().set_index('movieId')['title'].to_dict()

# Crear una matriz de interacciones para LightFM
user_ids = df['userId'].astype('category').cat.codes
item_ids = df['movieId'].astype('category').cat.codes
interactions = coo_matrix((df['rating'], (user_ids, item_ids)))

# Ajustar hiperparámetros para LightFM
lightfm_model = LightFM(loss='warp', no_components=150, learning_rate=0.05, item_alpha=1e-6, user_alpha=1e-6)
lightfm_model.fit(interactions, epochs=30, num_threads=2)

# Función para obtener las mejores recomendaciones para un usuario dado utilizando SVD
def get_top_recommendations_svd(user_id, n=rec):
    items = []
    trainset = data.build_full_trainset()
    items_not_rated_by_user = [item for item in range(1, trainset.n_items + 1) if item not in trainset.ur[user_id]]
    for item_id in items_not_rated_by_user:
        predicted_rating = best_model.predict(user_id, item_id).est
        items.append((item_id, predicted_rating))
    top_items = sorted(items, key=lambda x: x[1], reverse=True)[:n]
    return [(item[0], movie_titles.get(item[0], "Título no disponible")) for item in top_items]

# Función para obtener las mejores recomendaciones para un usuario dado utilizando LightFM
def get_top_recommendations_lightfm(user_id, n=rec):
    user_code = df['userId'].astype('category').cat.categories.get_loc(user_id)
    scores = lightfm_model.predict(user_code, np.arange(len(movie_titles)))
    top_items = np.argsort(-scores)[:n]
    return [(df['movieId'].astype('category').cat.categories[item], movie_titles.get(df['movieId'].astype('category').cat.categories[item], "Título no disponible")) for item in top_items]

# Ejemplo de uso, recomendación para el usuario dado
user_id = 1
top_recommendations_svd = get_top_recommendations_svd(user_id)
top_recommendations_lightfm = get_top_recommendations_lightfm(user_id)

print("Top recomendaciones para el usuario usando SVD:", user_id)
for rank, (movie_id, title) in enumerate(top_recommendations_svd, 1):
    print(f"{rank}. {title} (ID: {movie_id})")

print("\nTop recomendaciones para el usuario usando LightFM:", user_id)
for rank, (movie_id, title) in enumerate(top_recommendations_lightfm, 1):
    print(f"{rank}. {title} (ID: {movie_id})")

# Evaluar el modelo SVD con 5-fold cross-validation
print("\n\n\nEvaluación del modelo SVD:")
results_svd = cross_validate(best_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Evaluar el modelo LightFM
precision = precision_at_k(lightfm_model, interactions, k=rec).mean()
recall = recall_at_k(lightfm_model, interactions, k=rec).mean()
print("\nEvaluación del modelo LightFM:")
print(f"Precision at {rec}: {precision}")
print(f"Recall at {rec}: {recall}")

# Ajuste de hiperparámetros LightFM
param_grid_lightfm = {
    'no_components': [20, 50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'item_alpha': [1e-6, 1e-5, 1e-4],
    'user_alpha': [1e-6, 1e-5, 1e-4]
}

# Buscar los mejores hiperparámetros para LightFM
best_precision = 0
best_params = None
for no_components in param_grid_lightfm['no_components']:
    for learning_rate in param_grid_lightfm['learning_rate']:
        for item_alpha in param_grid_lightfm['item_alpha']:
            for user_alpha in param_grid_lightfm['user_alpha']:
                lightfm_model = LightFM(loss='warp', no_components=no_components, learning_rate=learning_rate, item_alpha=item_alpha, user_alpha=user_alpha)
                lightfm_model.fit(interactions, epochs=30, num_threads=2)
                precision = precision_at_k(lightfm_model, interactions, k=rec).mean()
                if precision > best_precision:
                    best_precision = precision
                    best_params = {
                        'no_components': no_components,
                        'learning_rate': learning_rate,
                        'item_alpha': item_alpha,
                        'user_alpha': user_alpha
                    }

print("\nMejores hiperparámetros para LightFM:")
print(best_params)
print(f"Mejor precisión: {best_precision}")

# Entrenar el mejor modelo LightFM
lightfm_model = LightFM(loss='warp', **best_params)
lightfm_model.fit(interactions, epochs=30, num_threads=2)

# Re-evaluar el mejor modelo LightFM
precision = precision_at_k(lightfm_model, interactions, k=rec).mean()
recall = recall_at_k(lightfm_model, interactions, k=rec).mean()
print("\nRe-evaluación del modelo LightFM con mejores hiperparámetros:")
print(f"Precision at {rec}: {precision}")
print(f"Recall at {rec}: {recall}")


Top recomendaciones para el usuario usando SVD: 1
1. Mother Night (ID: 893)
2. A Passage to India (ID: 7087)
3. Diabolique (ID: 7116)
4. Gladiator 1992 (ID: 8132)
5. Cinema Paradiso (ID: 1172)

Top recomendaciones para el usuario usando LightFM: 1
1. Time Bandits (ID: 2968)
2. Dracula (ID: 1339)
3. Mad Max 2: The Road Warrior (ID: 3703)
4. Gandhi (ID: 1293)
5. The Deer Hunter (ID: 1263)



Evaluación del modelo SVD:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8781  0.8742  0.8708  0.8744  0.8680  0.8731  0.0034  
MAE (testset)     0.6735  0.6728  0.6693  0.6718  0.6676  0.6710  0.0022  
Fit time          5.70    4.79    5.53    4.84    4.88    5.15    0.39    
Test time         0.12    0.12    0.18    0.13    0.19    0.15    0.03    

Evaluación del modelo LightFM:
Precision at 5: 0.8602086901664734
Recall at 5: 0.07120333077043094

Mejores hiperparámetros para LightFM:
{'no_compone

# **Entrega 4**


In [3]:
!pip install scikit-surprise
!pip install lightfm
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, GridSearchCV
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
from scipy.sparse import coo_matrix
import numpy as np

# URL del archivo CSV
data_url = "https://raw.githubusercontent.com/JossueGG/tec_bigdata_equipo17/main/ratings_small_entrega2.csv"

# Cargar los datos desde la URL
df = pd.read_csv(data_url)

# Rango de las calificaciones
reader = Reader(rating_scale=(1, 5))

# Número de recomendaciones a desplegar
rec = 5

# Crear un Dataset de Surprise
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Entrenando el modelo SVD
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Obtener el nombre de la película a partir de su ID
movie_titles = df[['movieId', 'title']].drop_duplicates().set_index('movieId')['title'].to_dict()

# Función para obtener las mejores recomendaciones para un usuario dado utilizando SVD
def get_top_recommendations_svd(user_id, n=rec):
    items = []
    items_not_rated_by_user = [item for item in range(1, trainset.n_items + 1) if item not in trainset.ur[user_id]]
    for item_id in items_not_rated_by_user:
        predicted_rating = model.predict(user_id, item_id).est
        items.append((item_id, predicted_rating))
    top_items = sorted(items, key=lambda x: x[1], reverse=True)[:n]
    return [(item[0], movie_titles.get(item[0], "Título no disponible")) for item in top_items]

# Búsqueda de hiperparámetros para SVD
param_grid = {
    'n_factors': [20, 50, 100, 150],
    'n_epochs': [10, 20, 30, 40, 50],
    'lr_all': [0.002, 0.005, 0.01, 0.02],
    'reg_all': [0.02, 0.05, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Mejor modelo SVD
best_model = gs.best_estimator['rmse']
best_model.fit(data.build_full_trainset())

# Crear una matriz de interacciones para LightFM
user_ids = df['userId'].astype('category').cat.codes
item_ids = df['movieId'].astype('category').cat.codes
interactions = coo_matrix((df['rating'], (user_ids, item_ids)))

# Ajustar hiperparámetros para LightFM
lightfm_model = LightFM(loss='warp', no_components=150, learning_rate=0.05, item_alpha=1e-6, user_alpha=1e-6)
lightfm_model.fit(interactions, epochs=30, num_threads=2)

# Función para obtener las mejores recomendaciones para un usuario dado utilizando LightFM
def get_top_recommendations_lightfm(user_id, n=rec):
    user_code = df['userId'].astype('category').cat.categories.get_loc(user_id)
    scores = lightfm_model.predict(user_code, np.arange(len(movie_titles)))
    top_items = np.argsort(-scores)[:n]
    return [(df['movieId'].astype('category').cat.categories[item], movie_titles.get(df['movieId'].astype('category').cat.categories[item], "Título no disponible")) for item in top_items]

# Ejemplo de uso, recomendación para el usuario dado
user_id = 1
top_recommendations_svd = get_top_recommendations_svd(user_id)
top_recommendations_lightfm = get_top_recommendations_lightfm(user_id)

print("Top recomendaciones para el usuario usando SVD:", user_id)
for rank, (movie_id, title) in enumerate(top_recommendations_svd, 1):
    print(f"{rank}. {title} (ID: {movie_id})")

print("\nTop recomendaciones para el usuario usando LightFM:", user_id)
for rank, (movie_id, title) in enumerate(top_recommendations_lightfm, 1):
    print(f"{rank}. {title} (ID: {movie_id})")

# Evaluar el modelo SVD con 5-fold cross-validation
print("\n\n\nEvaluación del modelo SVD:")
results_svd = cross_validate(best_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Evaluar el modelo LightFM
precision = precision_at_k(lightfm_model, interactions, k=rec).mean()
recall = recall_at_k(lightfm_model, interactions, k=rec).mean()
print("\nEvaluación del modelo LightFM:")
print(f"Precision at {rec}: {precision}")
print(f"Recall at {rec}: {recall}")

# Ajuste de hiperparámetros LightFM
param_grid_lightfm = {
    'no_components': [20, 50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'item_alpha': [1e-6, 1e-5, 1e-4],
    'user_alpha': [1e-6, 1e-5, 1e-4]
}

# Buscar los mejores hiperparámetros para LightFM
best_precision = 0
best_params = None
for no_components in param_grid_lightfm['no_components']:
    for learning_rate in param_grid_lightfm['learning_rate']:
        for item_alpha in param_grid_lightfm['item_alpha']:
            for user_alpha in param_grid_lightfm['user_alpha']:
                lightfm_model = LightFM(loss='warp', no_components=no_components, learning_rate=learning_rate, item_alpha=item_alpha, user_alpha=user_alpha)
                lightfm_model.fit(interactions, epochs=30, num_threads=2)
                precision = precision_at_k(lightfm_model, interactions, k=rec).mean()
                if precision > best_precision:
                    best_precision = precision
                    best_params = {
                        'no_components': no_components,
                        'learning_rate': learning_rate,
                        'item_alpha': item_alpha,
                        'user_alpha': user_alpha
                    }

print("\nMejores hiperparámetros para LightFM:")
print(best_params)
print(f"Mejor precisión: {best_precision}")

# Entrenar el mejor modelo LightFM
lightfm_model = LightFM(loss='warp', **best_params)
lightfm_model.fit(interactions, epochs=30, num_threads=2)

# Re-evaluar el mejor modelo LightFM
precision = precision_at_k(lightfm_model, interactions, k=rec).mean()
recall = recall_at_k(lightfm_model, interactions, k=rec).mean()
print("\nRe-evaluación del modelo LightFM con mejores hiperparámetros:")
print(f"Precision at {rec}: {precision}")
print(f"Recall at {rec}: {recall}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357253 sha256=eb83b1bbb87b9a0c66d5ab6884c78bf71847a807362472e47a8298d1e30880b5
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808333 sha256=b0a5193c39d5ae7d933755672198b485bea4245861cd22dc8d601a4b0fe628b0
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
Top recomendaciones para el usuario us